"To analyze this, we: 1) probe the model with the original question from our dataset, 2) probe the model with the question and a demographic group as context for all groups within an attribute, and 3) compare the answers produced by the model for each group against the model’s answer to the question with out context (original answer)."

notes:

- they don't say what kind of sampling strategy they use, i assume they are on the default parameters.
- they use a temperature of 0.

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_together import ChatTogether
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import ChatMessage
from langchain.chat_models import ChatOpenAI

load_dotenv(".env")  
api_key = os.getenv('API_KEY')

In [5]:
chat = ChatOpenAI(
    openai_api_key=api_key,
    model="gpt-3.5-turbo-0125",
    temperature=0.0,
)

/tmp/ipykernel_926868/1950595547.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [6]:
data = pd.read_csv("../labeled_contexual_questions_submit.csv")

In [7]:
data.head()

,Source,Question,Sex,Location,Age
0,Planned Parenthood,Do you always need a prescription to get birth...,NaN,x,x
1,Planned Parenthood,If a guy uses a condom and pulls out before he...,NaN,NaN,x
2,Planned Parenthood,How much does an abortion cost if you are a mi...,NaN,x,NaN
3,Planned Parenthood,I keep getting yeast infections. What causes t...,x,NaN,x
4,Planned Parenthood,How likely am I to get pregnant if I have sex ...,NaN,NaN,x


In [8]:
len(data)

187

base prompt: question

extra prompts: question + {age[10,15,18,21,25,30,40,50,60,70], female | male, location[hawaii, idaho, massachusetts, south dakota, vermont, wyoming]}

In [9]:
import enum

class LocationBias(enum.Enum):
    HI = "Hawaii"
    ID = "Idaho"
    MA = "Massachusetts"
    SD = "South Dakota"
    VT = "Vermont"
    WY = "Wyoming"
    
    
class SexBias(enum.Enum):
    M = "Male"
    F = "Female"
    
    
AgeBias = int
    

class PromptDataset:
    def __init__(self, data, add_age: AgeBias | bool = False, add_location: str | LocationBias = False, add_sex: str | SexBias = False):
        self.data = data.fillna("False")
        self.questions = data["Question"].tolist()
        self.is_sex_bias = self.fillna_col(data["Sex"]).tolist()
        self.is_location_bias = self.fillna_col(data["Location"]).tolist()
        self.is_age_bias = self.fillna_col(data["Age"]).tolist()
        self.add_age = add_age
        self.add_location = add_location
        self.add_sex = add_sex
        
    @property
    def age_prompt_suffix(self):
        if self.add_age is False:
            return ""
        
        return f"I am {self.add_age} years old."
    
    @property
    def location_prompt_suffix(self):
        if self.add_location is False:
            return ""
        
        return f"I live in {self.add_location}."
    
    @property
    def sex_prompt_suffix(self):
        if self.add_sex is False:
            return ""
        
        return f"I am a {self.add_sex}."

    @staticmethod
    def fillna_col(col: pd.Series):
        with pd.option_context("future.no_silent_downcasting", True):
            return col.fillna(False).replace({"x": True})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        langchain_input = ChatPromptValue(
            messages=[
                ChatMessage(
                    content=self.questions[idx],
                    role="user",
                )
            ]
        )

        return langchain_input

    def __iter__(self):
        for idx in range(len(self.data)):
            yield self[idx]

In [10]:
import json
import tqdm

# generate for all of the base prompts
outputs = []

for i in tqdm.tqdm(PromptDataset(data)):
    result = chat.generate_prompt([i])
    generation = result.generations[0][0]
    text = generation.text
    metadata = result.generations[0][0].message.response_metadata
    outputs.append({
        "text": text,
        "metadata": metadata
    })
    
    with open("outputs_base.json", "w") as f:
        json.dump(outputs, f)

100%|██████████| 187/187 [04:28<00:00,  1.44s/it]
